In [1]:
import pandas as pd
import numpy as np

In [57]:
train = pd.read_csv('data/processed_data.csv')
test = pd.read_csv('data/bicikelj_test.csv')

In [58]:
# Get list of stations
stations = []
for i in range(1, 84):
    stations.append(train.columns[i])

In [59]:
# For test set
def find_closest_smaller_row(df, timestamp, hour_offset):
    target_timestamp = pd.to_datetime(timestamp) - pd.DateOffset(hours=hour_offset)
    df["timestamp"] = pd.to_datetime(df["timestamp"])
            
    smaller_rows = df[df["timestamp"] <= target_timestamp]
    if smaller_rows.empty:
        return df.iloc[0,:]  # No smaller rows found
    closest_index = np.abs(smaller_rows["timestamp"] - target_timestamp).idxmin()
    closest_row = df.loc[closest_index]
    return closest_row

# c_row = find_closest_smaller_row(train, "timestamp", train.iloc[13], 1)

In [79]:
# Extract new features
import datetime


def get_time_based_features(timestamp):
    # From datetime64 get month, day, hour, dayOfWeek and isHoliday
    timestamp = np.datetime64(timestamp)
    dt_python = timestamp.astype(datetime.datetime)

    # Extract month, day, hour, and day_of_week
    month = dt_python.month
    day = dt_python.day
    hour = dt_python.hour
    minute = dt_python.minute
    day_of_week = dt_python.weekday()

    # Check for holidays
    is_holiday = 0
    if dt_python.month == 8 and dt_python.day in [15, 17]:
        is_holiday = 1
    elif dt_python.month == 9 and dt_python.day in [15, 23]:
        is_holiday = 1
   
    return month, day, hour, minute, day_of_week, is_holiday

In [80]:
test["month"], test["day"], test["hour"], test["minute"], test["day_of_week"], test["is_holiday"] = zip(*test["timestamp"].map(get_time_based_features))

C:\Users\sebas\AppData\Local\Temp\ipykernel_3016\4023795701.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test["month"], test["day"], test["hour"], test["minute"], test["day_of_week"], test["is_holiday"] = zip(*test["timestamp"].map(get_time_based_features))
C:\Users\sebas\AppData\Local\Temp\ipykernel_3016\4023795701.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test["month"], test["day"], test["hour"], test["minute"], test["day_of_week"], test["is_holiday"] = zip(*test["timestamp"].map(get_time_based_features))
C:\U

In [60]:
# Get n_bikes for each timestamp in test set
test_station = stations[3]
test_station

'CANKARJEVA UL.-NAMA'

In [61]:
test["timestamp"][0], train["timestamp"][0]

('2022-08-04 23:24:00', '2022-08-02 15:10:00')

In [70]:
# for station in stations:
#     test[f"{station}_2hr"] = test["timestamp"].apply(lambda time: find_closest_smaller_row(train, time, 2)[station])

In [73]:
days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]

for i, day in enumerate(days_of_week):
    test[day] = (train["day_of_week"] == i).astype(int)


hours = [i for i in range(24)]
for i in hours:
    test[f"hour_{i}"] = (train["hour"] == i).astype(int)

C:\Users\sebas\AppData\Local\Temp\ipykernel_3016\656407860.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[day] = (train["day_of_week"] == i).astype(int)
C:\Users\sebas\AppData\Local\Temp\ipykernel_3016\656407860.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[day] = (train["day_of_week"] == i).astype(int)
C:\Users\sebas\AppData\Local\Temp\ipykernel_3016\656407860.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

In [81]:
test.columns

Index(['timestamp', 'PREŠERNOV TRG-PETKOVŠKOVO NABREŽJE',
       'POGAČARJEV TRG-TRŽNICA', 'KONGRESNI TRG-ŠUBIČEVA ULICA',
       'CANKARJEVA UL.-NAMA', 'BREG', 'GRUDNOVO NABREŽJE-KARLOVŠKA C.',
       'MIKLOŠIČEV PARK', 'BAVARSKI DVOR', 'TRG OF-KOLODVORSKA UL.',
       ...
       'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month', 'day', 'hour',
       'minute', 'day_of_week', 'is_holiday'],
      dtype='object', length=287)

In [76]:
test.to_csv("data/test_expanded.csv", index=False)